In [1]:
!pip install transformers
!pip install transformers SentencePiece
from huggingface_hub import notebook_login
notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
correct_word = 'Delirium'
wrong_word = 'Deiliriuum'

## SGPT

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Muennighoff/SGPT-1.3B-weightedmean-nli-bitfit")
model = AutoModel.from_pretrained("Muennighoff/SGPT-1.3B-weightedmean-nli-bitfit").to(device)

model.eval()

GPTNeoModel(
  (wte): Embedding(50257, 2048)
  (wpe): Embedding(2048, 2048)
  (drop): Dropout(p=0.0, inplace=False)
  (h): ModuleList(
    (0-23): 24 x GPTNeoBlock(
      (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        )
      )
      (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
        (c_proj): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGE

In [15]:
correct_word_tokenized = tokenizer(correct_word, return_tensors='pt')
wrong_word_tokenized = tokenizer(wrong_word, return_tensors='pt')

def getEmbeddings(input_ids, attention_mask):
    # Get the embeddings
    with torch.no_grad():
        # Get hidden state of shape [bs, seq_len, hid_dim]
        last_hidden_state = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True).last_hidden_state

    # Get weights of shape [bs, seq_len, hid_dim]
    weights = (
        torch.arange(start=1, end=last_hidden_state.shape[1] + 1)
        .unsqueeze(0)
        .unsqueeze(-1)
        .expand(last_hidden_state.size())
        .float().to(last_hidden_state.device)
    )

    # Get attn mask of shape [bs, seq_len, hid_dim]
    input_mask_expanded = (
        attention_mask
        .unsqueeze(-1)
        .expand(last_hidden_state.size())
        .float()
    )

    # Perform weighted mean pooling across seq_len: bs, seq_len, hidden_dim -> bs, hidden_dim
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded * weights, dim=1)
    sum_mask = torch.sum(input_mask_expanded * weights, dim=1)

    embeddings = sum_embeddings / sum_mask

    return embeddings.cpu()

# Get embedding for correct word
input_ids = correct_word_tokenized['input_ids'].to(device)
attention_mask = correct_word_tokenized['attention_mask'].to(device)
correct_word_embedding = getEmbeddings(input_ids, attention_mask)

# Get embedding for wrong word
input_ids = wrong_word_tokenized['input_ids'].to(device)
attention_mask = wrong_word_tokenized['attention_mask'].to(device)
wrong_word_embedding = getEmbeddings(input_ids, attention_mask)

cos = 1 - cosine(correct_word_embedding[0], wrong_word_embedding[0])
print(cos)

0.7889525890350342


## BERT

In [8]:
from transformers import BertTokenizer, BertModel

In [9]:
tokenizer = BertTokenizer.from_pretrained("AIMH/mental-bert-large-uncased")
model = BertModel.from_pretrained("AIMH/mental-bert-large-uncased").to(device)

model.eval()

Some weights of the model checkpoint at AIMH/mental-bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at AIMH/mental-bert-large-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [10]:
correct_word_tokenized = tokenizer(correct_word, return_tensors='pt')
wrong_word_tokenized = tokenizer(wrong_word, return_tensors='pt')

input_ids_correct = correct_word_tokenized['input_ids'].to(device)
attention_mask_correct = correct_word_tokenized['attention_mask'].to(device)
token_type_ids_correct = correct_word_tokenized['token_type_ids'].to(device)

with torch.no_grad():
    embedding = model(input_ids=input_ids_correct,
                    attention_mask=attention_mask_correct,
                    token_type_ids=token_type_ids_correct)
    correct_word_embedding = embedding.last_hidden_state.mean(dim=1).cpu()

input_ids_wrong = wrong_word_tokenized['input_ids'].to(device)
attention_mask_wrong = wrong_word_tokenized['attention_mask'].to(device)
token_type_ids_wrong = wrong_word_tokenized['token_type_ids'].to(device)

with torch.no_grad():
    embedding = model(input_ids=input_ids_wrong,
                    attention_mask=attention_mask_wrong,
                    token_type_ids=token_type_ids_wrong)
    wrong_word_embedding = embedding.last_hidden_state.mean(dim=1).cpu()

cos = 1 - cosine(correct_word_embedding[0], wrong_word_embedding[0])
print(cos)

0.857570230960846


##XLNet

In [11]:
from transformers import XLNetTokenizer, XLNetModel

In [12]:
tokenizer = XLNetTokenizer.from_pretrained('AIMH/mental-xlnet-base-cased')
model = XLNetModel.from_pretrained('AIMH/mental-xlnet-base-cased').to(device)
model.eval()

Some weights of the model checkpoint at AIMH/mental-xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLNetModel(
  (word_embedding): Embedding(32000, 768)
  (layer): ModuleList(
    (0-11): 12 x XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, bias=True)
        (layer_2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (activation_function): GELUActivation()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [13]:
correct_word_tokenized = tokenizer(correct_word, return_tensors='pt')
wrong_word_tokenized = tokenizer(wrong_word, return_tensors='pt')

input_ids_correct = correct_word_tokenized['input_ids'].to(device)
attention_mask_correct = correct_word_tokenized['attention_mask'].to(device)
token_type_ids_correct = correct_word_tokenized['token_type_ids'].to(device)

with torch.no_grad():
    embedding = model(input_ids=input_ids_correct,
                    attention_mask=attention_mask_correct,
                    token_type_ids=token_type_ids_correct)
    correct_word_embedding = embedding.last_hidden_state.mean(dim=1).cpu()

input_ids_wrong = wrong_word_tokenized['input_ids'].to(device)
attention_mask_wrong = wrong_word_tokenized['attention_mask'].to(device)
token_type_ids_wrong = wrong_word_tokenized['token_type_ids'].to(device)

with torch.no_grad():
    embedding = model(input_ids=input_ids_wrong,
                    attention_mask=attention_mask_wrong,
                    token_type_ids=token_type_ids_wrong)
    wrong_word_embedding = embedding.last_hidden_state.mean(dim=1).cpu()

cos = 1 - cosine(correct_word_embedding[0], wrong_word_embedding[0])
print(cos)

0.9252498149871826
